In [2]:
import torch
from torchvision import datasets, transforms, models
import numpy as np
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 1033120000
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import helper
from tqdm import tqdm

from tensorboardX import SummaryWriter
from trains import Task
import warnings
warnings.filterwarnings("ignore")

task = Task.init(project_name = 'Valid ID Classify',
                task_name = 'Stage 2 - MobileNet - Pytorch - sample')

In [45]:
writer = SummaryWriter()

# define transformation for images that are put into the dataset on here
transform = transforms.Compose([transforms.Resize(225),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# importing our data using torchvision.datasets.
train_set = datasets.ImageFolder("sample/Train", transform = transform)
test_set = datasets.ImageFolder("sample/Valid", transform = transform)

# put data into a Dataloader using torch
train_loader = torch.utils.data.DataLoader(train_set, batch_size= 64,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size= 64, shuffle = True)

In [46]:
import torch.nn as nn
import torch.optim as optim
# Using a pretrained model
model = models.mobilenet_v2(pretrained = True)

# turn off training for their parameters
for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(nn.Dropout(0.2, inplace = False),
              nn.Linear(1280, 512, bias = True),
              nn.Dropout(0.2, inplace = False),
              nn.Linear(512, 3, bias = True))

model.classifier = classifier

2020-06-12 15:11:56,352 - trains.model - INFO - Selected model id: c45e2ee1aa78473aad02c22e7b18ac80


In [47]:
# Find the device available to use using torch library
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move model to the device specified above
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [48]:
# Set the error function using torch.nn as nn library
criterion = nn.CrossEntropyLoss()

# Set the optimizer function using torch.optim as optim library
optimizer = optim.SGD(model.classifier.parameters(), lr = 0.01, momentum=0.9)

In [49]:
global model_name
model_name = 0

In [50]:
def train(epochs):
    for epoch in range(epochs):
        train_loss = 0
        test_loss = 0
        accuracy = 0
        
        # Training the model
        model.train()
        for inputs, labels in tqdm(train_loader):
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Clear optimizers
            optimizer.zero_grad()
            # Forward pass
            output = model.forward(inputs)
            # Loss
            loss = criterion(output, labels)
            # Calculate gradients (backpropogation)
            loss.backward()
            # Adjust parameters based on gradients
            optimizer.step()
            # Add the loss to the training set's rnning loss
            train_loss += loss.item()*inputs.size(0)
        
        global model_name
        torch.save(model, 'sample_pytorch_model/' + str(model_name) + '_pytorch_v1.pth')

        model_name += 1

        # Evaluating the model
        model.eval()
        # Tell torch not to calculate gradients
        with torch.no_grad():
            for inputs, labels in tqdm(test_loader):
                # Move to device
                inputs, labels = inputs.to(device), labels.to(device)
                # Forward pass
                output = model.forward(inputs)
                # Calculate Loss
                testloss = criterion(output, labels)
                # Add loss to the validation set's running loss
                test_loss += testloss.item()*inputs.size(0)
                
                # Since our model outputs a LogSoftmax, find the real 
                # percentages by reversing the log function
                output = torch.exp(output)
                # Get the top class of the output
                top_p, top_class = output.topk(1, dim=1)
                # See how many of the classes were correct?
                equals = top_class == labels.view(*top_class.shape)
                # Calculate the mean (get the accuracy for this batch)
                # and add it to the running accuracy for this epoch
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()            
                
        
        # Get the average loss for the entire epoch
        train_loss = train_loss/len(train_loader.dataset)
        test_loss = test_loss/len(test_loader.dataset)
        
        writer.add_scalar('data/train loss', train_loss, epoch)
        writer.add_scalar('data/valid loss', test_loss, epoch)
        writer.add_scalar('data/accuracy', accuracy/len(test_loader), epoch)
        # Print out the information
        print('Accuracy: ', accuracy/len(test_loader))
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, test_loss))

In [ ]:
train(5)



  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:01<00:30,  1.59s/it]

 10%|█         | 2/20 [00:03<00:30,  1.69s/it]

 15%|█▌        | 3/20 [00:04<00:26,  1.53s/it]

 20%|██        | 4/20 [00:05<00:23,  1.46s/it]

 25%|██▌       | 5/20 [00:07<00:21,  1.42s/it]

 30%|███       | 6/20 [00:09<00:23,  1.70s/it]

 35%|███▌      | 7/20 [00:11<00:22,  1.72s/it]

 40%|████      | 8/20 [00:14<00:24,  2.00s/it]

 45%|████▌     | 9/20 [00:16<00:23,  2.15s/it]

 50%|█████     | 10/20 [00:18<00:21,  2.13s/it]

 55%|█████▌    | 11/20 [00:20<00:18,  2.01s/it]

 60%|██████    | 12/20 [00:22<00:15,  1.94s/it]

 65%|██████▌   | 13/20 [00:24<00:13,  1.91s/it]

 70%|███████   | 14/20 [00:27<00:13,  2.25s/it]

 75%|███████▌  | 15/20 [00:28<00:10,  2.06s/it]